In [1]:
import pandas as pd

Changing column names and rounding of USWTDB to match NREL datasite

In [ ]:
uswtdb = pd.read_csv('uswtdb_V8_1_20250522.csv')
uswtdb['long_rounded'] = uswtdb['xlong'].round(3)
uswtdb['lat_rounded'] = uswtdb['ylat'].round(3)
uswtdb.head()

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_retro_yr,t_offshore,t_conf_atr,t_conf_loc,t_img_date,t_img_src,xlong,ylat,longtude,latitude
0,3119490,40-097498,2021-WTW-7911-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,NaN,0,3,3,1/4/2023,Maxar,-99.741096,36.423683,-99.7411,36.4237
1,3124158,40-098166,2021-WTW-7917-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,NaN,0,3,3,1/4/2023,Maxar,-99.796494,36.503357,-99.7965,36.5034
2,3123358,40-098173,2021-WTW-7871-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,NaN,0,3,3,1/4/2023,Maxar,-99.741119,36.502590,-99.7411,36.5026
3,3123761,40-097539,2021-WTW-7888-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,NaN,0,3,3,1/4/2023,Maxar,-99.771027,36.456665,-99.7710,36.4567
4,3124023,40-097763,2021-WTW-7894-OE,NaN,65511.0,OK,Ellis County,40045.0,25 Mile Creek,2022.0,...,NaN,0,3,3,1/4/2023,Maxar,-99.775742,36.445465,-99.7757,36.4455


Changing column names and rounding of USWTDB to match NREL datasite

In [ ]:
nrel = pd.read_csv('wtk_site_metadata.csv')
nrel['long_rounded'] = nrel['longitude'].round(3)
nrel['lat_rounded'] = nrel['latitude'].round(3)
nrel.head()


,site_id,longitude,latitude,State,County,fraction_of_usable_area,power_curve,capacity,wind_speed,capacity_factor,full_timeseries_directory,full_timeseries_path,long_rounded,lat_rounded
0,0,-117.147260,23.510410,Unknown,Unknown,1.0,offshore,16,6.07,0.169,0,0/0.nc,-117.1473,23.5104
1,1,-93.946777,24.007446,Unknown,Unknown,1.0,offshore,16,7.43,0.302,0,0/1.nc,-93.9468,24.0074
2,2,-97.482483,25.069138,Unknown,Unknown,1.0,offshore,16,8.19,0.375,0,0/2.nc,-97.4825,25.0691
3,3,-97.463135,25.069443,Unknown,Unknown,1.0,offshore,16,8.19,0.375,0,0/3.nc,-97.4631,25.0694
4,4,-97.443756,25.069763,Unknown,Unknown,1.0,offshore,16,8.19,0.376,0,0/4.nc,-97.4438,25.0698


I'll probably use a library(openball from sklearn) to map each wind turbine to the nearest grid on the NERL dataset
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree


In [18]:
%pip install geopandas

import geopandas as gpd
from shapely.geometry import Point

# Build geometry columns
g_uswtdb = gpd.GeoDataFrame(
    uswtdb,
    geometry=gpd.points_from_xy(uswtdb.xlong, uswtdb.ylat),
    crs="EPSG:4326"
)

g_nrel = gpd.GeoDataFrame(
    nrel,
    geometry=gpd.points_from_xy(nrel.longitude, nrel.latitude),
    crs="EPSG:4326"
)

g_uswtdb_m = g_uswtdb.to_crs(3857)
g_nrel_m = g_nrel.to_crs(3857)

# 4) One tolerance for everyone (tune this after inspecting a distance histogram)
MAX_DIST_M = 25000  # 25 km

# 5) Nearest-neighbor spatial join with a distance cap

joined_m = gpd.sjoin_nearest(
    g_uswtdb_m,
    g_nrel_m[[
        "geometry",
        "site_id",
        "longitude",
        "latitude",
        "State",
        "County",
        "fraction_of_usable_area",
        "power_curve",
        "capacity",
        "wind_speed",
        "capacity_factor",
        "full_timeseries_directory",
        "full_timeseries_path"
    ]],
    how="left",
    distance_col="dist_m",
    max_distance=MAX_DIST_M,
)


joined_m["match_ok"] = joined_m["site_id"].notna()

joined_m.to_csv("uswt_nrel_joined.csv", index=False)

Note: you may need to restart the kernel to use updated packages.
